## Getting inspiration from the modular addition video of neel nanda

### Plan of attack
- Get the adjectives embeddings
- Get the embeddings
- Compute the mean of the embeddings
- Substract the mean of the embeddings
- Compute the svd matrix
- Plot the singular values

In [1]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

from IPython import get_ipython
ipython = get_ipython()
# Code to automatically update the HookedTransformer code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

pio.renderers.default = "vscode"
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader
from jaxtyping import Float, Int

from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
torch.set_grad_enabled(False)
from neel_plotly import line, imshow, scatter
import transformer_lens.patching as patching



import plotly.io as pio
pio.renderers.default = "vscode"


def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)


def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

/tmp/ipykernel_307894/4193134732.py:7: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_307894/4193134732.py:8: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
#Number of tokens for digit
import matplotlib.pyplot as plt
import json
with open("datasets/adj_nouns_lists.json","r") as f:
    adj_nouns_list=json.load(f)
pos_adj=[" "+pa[0] for pa in adj_nouns_list['adjective_pairs']]
neg_adj=[" "+pa[1] for pa in adj_nouns_list['adjective_pairs']]



In [6]:
import matplotlib.pyplot as plt

model = HookedTransformer.from_pretrained("gpt2-small")

# Prepare the data
pos_adj_index=torch.tensor(([model.to_single_token(i) for i in pos_adj]))
pos_adj_embeddings=model.W_E[pos_adj_index].float()
i_am_a_positive_adj=pos_adj_embeddings.mean(0)
pos_adj_embeddings_centered=pos_adj_embeddings-i_am_a_positive_adj
U,S,Vh=pos_adj_embeddings_centered.svd()
line(S,title="Singular value of W_E")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
# Prepare the data
neg_adj_index=torch.tensor(([model.to_single_token(i) for i in neg_adj]))
neg_adj_embeddings=model.W_E[neg_adj_index].float()
i_am_a_negative_adj=neg_adj_embeddings.mean(0)
neg_adj_embeddings_centered=neg_adj_embeddings-i_am_a_negative_adj
U,S,Vh=neg_adj_embeddings_centered.svd()
line(S,title="Singular value of W_E")